In [4]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier

import time

ModuleNotFoundError: No module named 'art'

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',
            # 'Macro AUC': '',
            # 'Weighted AUC': '',
            'TPR': '',
            'FNR': '',
            'TNR': '',
            'FPR': '',
        }
head = pd.DataFrame([head])
head.to_csv("/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/attackmodel.csv", mode='a', index=False)




In [3]:
def calculate_performance_metrics(X_test, y_true, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        # macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        # weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
            # 'Macro AUC': macro_auc,
            # 'Weighted AUC': weighted_auc,
            'TPR': TPR_macro,
            'FNR': FNR_macro,
            'TNR': TNR_macro,
            'FPR': FPR_macro,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/attackmodel.csv", mode='a', index=False, header=False)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [6]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_val.npy')

In [8]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2, 3, 4]), array([    42,     52,  15661,   1648, 221350]))

In [6]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [7]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [8]:
model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/dnn_pytorch.pt"))

<All keys matched successfully>

In [9]:
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

In [10]:
    
classifier = PyTorchClassifier(
    model=model,
    clip_values=(-5, 5),
    loss=nn.CrossEntropyLoss(),
    optimizer=optim.Adam(model.parameters(), lr=0.001),
    input_shape=(input_shape,),
    nb_classes=output_shape,
    device_type='gpu'
)

In [11]:
from art.attacks.evasion import SimBA, SpatialTransformation, DeepFool, BasicIterativeMethod, FastGradientMethod, ProjectedGradientDescent


In [12]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = BasicIterativeMethod(classifier, eps=epsilon, max_iter = 10, verbose=False) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [13]:
for epsilon in epsilon_values:
    adv_crafter = FastGradientMethod(classifier, eps=epsilon) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [15]:
for epsilon in epsilon_values:
    adv_crafter = ProjectedGradientDescent(classifier, eps=epsilon, verbose=False, max_iter=10) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

In [18]:
DF_eps = [0.01, 0.1, 0.2, 0.3]
for epsilon in DF_eps:
    adv_crafter = DeepFool(classifier, epsilon=epsilon, verbose=False, max_iter=10) # init eps=1e-06
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)


In [ ]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

DF_eps = [1e-07, 1e-06, 1e-05, 1e-04]
for epsilon in DF_eps:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

Accuracy: 0.9999581157095409

macro
Precision: 0.9760275644517383
Recall: 0.9711603118287865
F1 Score: 0.971746830391899

weighted
Precision: 0.9999615942542808
Recall: 0.9999581157095409
F1 Score: 0.9999580733593396

Mean FNR: 0.02883968817121358
Mean TNR: 0.9999809556341834
Mean FPR: 1.9044365816625028e-05
Mean TPR: 0.9711603118287865
Accuracy: 0.9996774909634644

macro
Precision: 0.8196007792102954
Recall: 0.9389366493411517
F1 Score: 0.8659194698907322

weighted
Precision: 0.9997748264873905
Recall: 0.9996774909634644
F1 Score: 0.9997128387808352

Mean FNR: 0.06106335065884842
Mean TNR: 0.9999245569611481
Mean FPR: 7.544303885183419e-05
Mean TPR: 0.9389366493411517
Accuracy: 0.3404061938488731

macro
Precision: 0.6901771545662203
Recall: 0.7501413861932366
F1 Score: 0.5889055813084352

weighted
Precision: 0.9936338768020678
Recall: 0.3404061938488731
F1 Score: 0.48843781530459945

Mean FNR: 0.24985861380676347
Mean TNR: 0.8653903530704061
Mean FPR: 0.13460964692959376
Mean TPR: 0.7

In [11]:
from art.attacks.evasion import ZooAttack

epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    adv_crafter = ZooAttack(classifier, confidence=epsilon, max_iter = 1, batch_size=1, nb_parallel=20) # init eps=0.3
    x_test_adv = adv_crafter.generate(x=x_test, y=y_test)

    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    np.save(filename, x_test_adv)

ZOO: 100%|██████████| 238753/238753 [49:37<00:00, 80.18it/s]


In [12]:
epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]

# Iterate over epsilon values
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ZOO', epsilon)

Accuracy: 0.9996356066730051

macro
Precision: 0.8078757578080079
Recall: 0.9171799071202263
F1 Score: 0.849528393703164

weighted
Precision: 0.9997414349559645
Recall: 0.9996356066730051
F1 Score: 0.9996727356218181

Mean FNR: 0.08282009287977363
Mean TNR: 0.9999155142831192
Mean FPR: 8.44857168808971e-05
Mean TPR: 0.9171799071202263
Accuracy: 0.9996607372472807

macro
Precision: 0.815319048907555
Recall: 0.9179080624600321
F1 Score: 0.855366124441364

weighted
Precision: 0.999752740413736
Recall: 0.9996607372472807
F1 Score: 0.9996931322660902

Mean FNR: 0.0820919375399678
Mean TNR: 0.9999206000104313
Mean FPR: 7.9399989568862e-05
Mean TPR: 0.9179080624600321
Accuracy: 0.9996691141053725

macro
Precision: 0.8200523179297734
Recall: 0.918150780906634
F1 Score: 0.8586949010772369

weighted
Precision: 0.9997546951270687
Recall: 0.9996691141053725
F1 Score: 0.9996995919158222

Mean FNR: 0.08184921909336586
Mean TNR: 0.9999222172294828
Mean FPR: 7.778277051703603e-05
Mean TPR: 0.918150780

In [ ]:
epsilon_values = [0.01, 0.1, 0.2, 0.3]

print("start BIM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

print("start FGSM")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_FGSM_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'FGSM', epsilon)

print("start PGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_PGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'PGD', epsilon)

print("start DF")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_DF_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'DF', epsilon)

print("start AutoPGD")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_AutoPGD_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'AutoPGD', epsilon)


print("start BA")
for epsilon in epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_BA_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BA', epsilon)



print("start ZOO")
ZOO_epsilon_values = [0, 0.01, 0.1, 0.2, 0.3]
# Iterate over epsilon values
for epsilon in ZOO_epsilon_values:
    filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_ZOO_eps_{epsilon}.npy'
    x_test_adv = np.load(filename)

    calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'ZOO', epsilon)

In [13]:
print("start HopSkipJump")
filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_HopSkipJump_eps_0.npy'
x_test_adv = np.load(filename)
calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'HopSkipJump', '0')

start HopSkipJump
Accuracy: 0.4186418599975707

macro
Precision: 0.20919562775934503
Recall: 0.24238239529507632
F1 Score: 0.14602162864375154

weighted
Precision: 0.888392280498441
Recall: 0.4186418599975707
F1 Score: 0.5632225135723031

Mean FNR: 0.7576176047049237
Mean TNR: 0.8297579304692592
Mean FPR: 0.17024206953074086
Mean TPR: 0.24238239529507632
